In [1]:
import pandas as pd
import math

In [2]:
df = pd.read_csv("Data/ACS_2019_data.csv")

C:\Users\bhatt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [3]:
# Dropping Margin of Error and non-uninsured, non-percentage columns, excluding id and geographic area name
for column in df.columns:
    actual_col_name = df.loc[0][column]
    is_id = (actual_col_name == 'id' or actual_col_name == 'Geographic Area Name')
    uninsured_and_percent = "Percent Uninsured" in actual_col_name
    margin_of_error = ("Margin of Error" in actual_col_name or "margin of error" in actual_col_name)
    if (margin_of_error or not uninsured_and_percent) and not is_id:
        df = df.drop(columns=column)

In [4]:
# Creating meta dataframe
meta_df = pd.read_csv('Data/ACS_2019_meta_data.csv')


In [5]:
# Dropping Margin of Error columns from meta_df or any non-uninsured columns
for index in meta_df.index:
    uninsured_and_percent = "Percent Uninsured" in meta_df.loc[index]['id']
    margin_of_error = ("Margin of Error" in meta_df.loc[index]['id'] or "margin of error" in meta_df.loc[index]['id'])
    if margin_of_error or not uninsured_and_percent:
        meta_df = meta_df.drop(index)

In [6]:
def underscore_adder(cleaned_string, key_word):
    """
    This is a helper function for cleanup(). 
    """
    if key_word in cleaned_string:
        temp_split = cleaned_string.split("_")
        for i in range(len(temp_split)):
            temp_split_copy = temp_split
            #Booleans
            key_word_present = key_word in temp_split[i]
            already_has_underscore = key_word+"_" in temp_split[i]
            word_is_poverty = "poverty" in temp_split[i]
            last_word_is_key = (i == len(temp_split)-1) and (key_word == temp_split[i])
            
            if key_word_present and not already_has_underscore and not word_is_poverty and not last_word_is_key:
                new_key_word = key_word + "_"
                temp_split_copy[i] = temp_split_copy[i].replace(key_word, new_key_word)

        cleaned_string = "_".join([str(item) for item in temp_split_copy])
    return cleaned_string

In [7]:
# Now, making id data frame more legible using this function
def cleanup(string):
    """
    Function returns legible `id` columns for `meta_df`. `meta_df` will eventually be used to create a dict
    that renames columns in df
    
    Arguments:
    `string`: a string datatype coming from `id` column of `meta_df`
    
    Outputs:
    `cleaned_string`: cleaned version of `string`
    """
    cleaned_string = string.lower()
    # Replacing spaces with "_"
    cleaned_string = cleaned_string.replace(" ","_")
    
    if "!!" in string:
        # Taking out descriptive filler in the beginning
        string_list = cleaned_string.split("!!")[1:]
        cleaned_string = ""
        
        for substr in string_list:
            cleaned_string += substr
        
        # Removing 'civilian_noninsitutionalized_population'
        if 'civilian_noninstitutionalized_' in cleaned_string:
            cleaned_string = cleaned_string.replace("civilian_noninstitutionalized_", "")
        
        # Add underscore after uninsured, and other words where neccessary
        key_word_list = ["uninsured", "population", "months", "determined", "age", "population", "sex",
                          "origin", "status", "attainment", "households", "experience", "years", "families", "born",
                          "over", "force", "arrangements"]
        for key_word in key_word_list:
            cleaned_string = underscore_adder(cleaned_string, key_word)
        
        # Strip commas, apostrophes, etc.
        remove_char = "',"
        for char in remove_char:
            cleaned_string = cleaned_string.replace(char, "")
        
        # Make sure all underscores are singular
        cleaned_string = cleaned_string.replace("__", "_").replace("___", "_").replace("-","_").replace('__', '_')
        cleaned_string = cleaned_string.replace(")", ")_")
        
        
        
            

    return cleaned_string

In [8]:
# Apply cleanup() function to meta_df id column
meta_df['id'] = meta_df['id'].apply(cleanup)

In [9]:
# Set 'GEO_ID' as index
meta_df = meta_df.reset_index()
meta_df = meta_df.set_index(['GEO_ID'])

In [10]:
# Create dict to rename columns with
rename_dict = meta_df.to_dict()
rename_dict = rename_dict['id']
rename_dict['GEO_ID'] = 'GEO_ID'
rename_dict['NAME'] = 'NAME'

In [11]:
# Drop redundant column from df
df = df.rename(columns = rename_dict)
df = df.drop(0)

In [12]:
df.dtypes

GEO_ID                                                                                                                                                                                   object
NAME                                                                                                                                                                                     object
percent_uninsured_population                                                                                                                                                             object
percent_uninsured_population_age_under_6_years                                                                                                                                           object
percent_uninsured_population_age_6_to_18_years                                                                                                                                           object
                                        

In [13]:
for column in df.columns:
    print(column)

GEO_ID
NAME
percent_uninsured_population
percent_uninsured_population_age_under_6_years
percent_uninsured_population_age_6_to_18_years
percent_uninsured_population_age_19_to_25_years
percent_uninsured_population_age_26_to_34_years
percent_uninsured_population_age_35_to_44_years
percent_uninsured_population_age_45_to_54_years
percent_uninsured_population_age_55_to_64_years
percent_uninsured_population_age_65_to_74_years
percent_uninsured_population_age_75_years_and_older
percent_uninsured_population_age_under_19_years
percent_uninsured_population_age_19_to_64_years
percent_uninsured_population_age_65_years_and_older
percent_uninsured_population_sex_male
percent_uninsured_population_sex_female
percent_uninsured_population_race_and_hispanic_or_latino_origin_white_alone
percent_uninsured_population_race_and_hispanic_or_latino_origin_black_or_african_american_alone
percent_uninsured_population_race_and_hispanic_or_latino_origin_american_indian_and_alaska_native_alone
percent_uninsured_popul

In [14]:
# Convert GEO_ID and NAME to category
df['GEO_ID'] = df['GEO_ID'].astype('category')
df['NAME'] = df['NAME'].astype('category')

In [15]:
def string_to_flt(string):
    if string=="-":
        return string
    else:
        return float(string)

In [16]:
# Convert float values to string
for column in df.columns.to_list()[2:]:
    df[column]= df[column].apply(string_to_flt)

In [17]:
df.set_index('GEO_ID')

,NAME,percent_uninsured_population,percent_uninsured_population_age_under_6_years,percent_uninsured_population_age_6_to_18_years,percent_uninsured_population_age_19_to_25_years,percent_uninsured_population_age_26_to_34_years,percent_uninsured_population_age_35_to_44_years,percent_uninsured_population_age_45_to_54_years,percent_uninsured_population_age_55_to_64_years,percent_uninsured_population_age_65_to_74_years,...,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_under_$25000,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$25000_to_$49999,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$50000_to_$74999,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$75000_to_$99999,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$100000_and_over,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_below_138_percent_of_the_poverty_threshold,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_138_to_399_percent_of_the_poverty_threshold,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_at_or_above_400_percent_of_the_poverty_threshold,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_below_100_percent_of_the_poverty_threshold
GEO_ID,,,,,,,,,,,,,,,,,,,,,
0500000US01001,"Autauga County, Alabama",7.1,1.7,1.8,13.3,16.1,14.3,5.8,7.8,0.0,...,13.2,10.3,5.3,5.1,3.6,7.1,15.4,6.1,3.1,17.2
0500000US01003,"Baldwin County, Alabama",8.9,2.2,4.4,19.5,20.8,13.2,12.2,8.5,0.5,...,18.5,12.0,9.4,7.9,3.3,8.8,19.2,10.4,2.8,20.7
0500000US01005,"Barbour County, Alabama",11.3,3.4,3.2,29.4,26.2,20.4,14,9.9,0.3,...,16.4,14.3,3.3,7.2,3.5,11.3,18.5,8.4,2.9,18.2
0500000US01007,"Bibb County, Alabama",10.7,4.5,0.8,21.9,21.5,17.7,18.6,6.9,0.0,...,19.6,10.6,7.6,7.8,6.1,10.7,21.2,8.9,2.1,18.9
0500000US01009,"Blount County, Alabama",10.8,6.1,5.8,15.7,18.1,23.8,12.8,10.6,0.2,...,18.4,17.8,7.0,6.3,4.2,10.7,22.7,9.2,3.6,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0500000US56037,"Sweetwater County, Wyoming",11.3,13.8,7.1,16.5,19.6,12.3,14.3,9.6,0.4,...,28.1,17.3,14.0,10.0,3.6,11.3,32.9,10.8,4.2,35.1
0500000US56039,"Teton County, Wyoming",12.7,5.9,11.8,19.2,26.8,15.7,6.5,10.0,0.0,...,10.9,31.3,19.6,17.6,4.4,12.7,26.8,23.0,3.6,22.7
0500000US56041,"Uinta County, Wyoming",11.2,1.8,8.9,28.4,15.0,18.5,14.6,9.5,0.9,...,16.2,17.9,12.8,5.6,7.7,11.2,19.5,13.4,4.3,18.7


In [18]:
# Create a state column
df['state'] = df['NAME'].apply(lambda x: x.split(",")[1].strip(" "))
df['state'] = df['state'].astype('category')

In [19]:
df.to_csv("Data/Cleaned_ACS.csv")

In [20]:
df.head()

,GEO_ID,NAME,percent_uninsured_population,percent_uninsured_population_age_under_6_years,percent_uninsured_population_age_6_to_18_years,percent_uninsured_population_age_19_to_25_years,percent_uninsured_population_age_26_to_34_years,percent_uninsured_population_age_35_to_44_years,percent_uninsured_population_age_45_to_54_years,percent_uninsured_population_age_55_to_64_years,...,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$25000_to_$49999,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$50000_to_$74999,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$75000_to_$99999,percent_uninsured_population_household_income_(in_2019_inflation_adjusted_dollars)_total_household_population_$100000_and_over,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_below_138_percent_of_the_poverty_threshold,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_138_to_399_percent_of_the_poverty_threshold,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_at_or_above_400_percent_of_the_poverty_threshold,percent_uninsured_population_ratio_of_income_to_poverty_level_in_the_past_12_months_population_for_whom_poverty_status_is_determined_below_100_percent_of_the_poverty_threshold,state
1,0500000US01001,"Autauga County, Alabama",7.1,1.7,1.8,13.3,16.1,14.3,5.8,7.8,...,10.3,5.3,5.1,3.6,7.1,15.4,6.1,3.1,17.2,Alabama
2,0500000US01003,"Baldwin County, Alabama",8.9,2.2,4.4,19.5,20.8,13.2,12.2,8.5,...,12.0,9.4,7.9,3.3,8.8,19.2,10.4,2.8,20.7,Alabama
3,0500000US01005,"Barbour County, Alabama",11.3,3.4,3.2,29.4,26.2,20.4,14,9.9,...,14.3,3.3,7.2,3.5,11.3,18.5,8.4,2.9,18.2,Alabama
4,0500000US01007,"Bibb County, Alabama",10.7,4.5,0.8,21.9,21.5,17.7,18.6,6.9,...,10.6,7.6,7.8,6.1,10.7,21.2,8.9,2.1,18.9,Alabama
5,0500000US01009,"Blount County, Alabama",10.8,6.1,5.8,15.7,18.1,23.8,12.8,10.6,...,17.8,7.0,6.3,4.2,10.7,22.7,9.2,3.6,22.0,Alabama
